Hosting MCP Server on Amazon Bedrock AgentCore Runtime - OAuth Inbound Authentication

In [1]:
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore_starter_toolkit.operations.runtime import destroy_bedrock_agentcore
from boto3.session import Session
from pathlib import Path
import os

In [2]:
boto_session = Session()
region = boto_session.region_name

agentcore_control_client = boto_session.client("bedrock-agentcore-control", region_name=region)
ssm_client = boto_session.client('ssm', region_name=region)

tool_name = "mcp_server_iam"

Understanding MCP (Model Context Protocol)
MCP is a protocol that allows AI models to securely access external data and tools. Key concepts:

Tools: Functions that the AI can call to perform actions  
Streamable HTTP: Transport protocol used by AgentCore Runtime  
Session Isolation: Each client gets isolated sessions via Mcp-Session-Id header  
Stateless Operation: Servers must support stateless operation for scalability  
AgentCore Runtime expects MCP servers to be hosted on 0.0.0.0:8000/mcp as the default path.  

Creating MCP Server

In [3]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

# stateless_http=True: Required for AgentCore Runtime compatibility
mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers together"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """Greet a user by name"""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

Overwriting mcp_server.py


Creating Local Testing Client

In [4]:
%%writefile mcp_client_local.py
import asyncio

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    # it calls the function returns the 3 values for read_stream, write_stream, and 
    # the one you dont care then due to with for cleanup
    # in normal sync we use  with open("file.txt") as f:
    # in here it also uses destructor to get the values as read_stream and write_stream
    async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

Overwriting mcp_client_local.py


Configuring AgentCore Runtime Deployment

In [3]:
print(f"Using AWS region: {region}")

required_files = ["mcp_server.py", "pyproject.toml"]
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="pyproject.toml",
    region=region,
    protocol="MCP",
    agent_name=tool_name,
)
print("Configuration completed ✓")

Entrypoint parsed: file=/home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/hosting-mcp-server/mcp_server.py, bedrock_agentcore_name=mcp_server
Configuring BedrockAgentCore agent: mcp_server_iam


Using AWS region: ap-southeast-2
All required files found ✓
Configuring AgentCore Runtime...


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated .dockerignore
Generated Dockerfile: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/hosting-mcp-server/Dockerfile
Generated .dockerignore: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/hosting-mcp-server/.dockerignore
Setting 'mcp_server_iam' as default agent
Bedrock AgentCore configured: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/hosting-mcp-server/.bedrock_agentcore.yaml


Configuration completed ✓


In [8]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'mcp_server_iam' to account 381491838394 (ap-southeast-2)
Setting up AWS resources (ECR repository, execution roles)...
Using ECR repository from config: 381491838394.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-mcp_server_iam
Using execution role from config: arn:aws:iam::381491838394:role/AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-22e9392f39
Preparing CodeBuild project and uploading source...


Launching MCP server to AgentCore Runtime...
This may take several minutes...


Getting or creating CodeBuild execution role for agent: mcp_server_iam
Role name: AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-22e9392f39
Reusing existing CodeBuild execution role: arn:aws:iam::381491838394:role/AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-22e9392f39
Using .dockerignore with 44 patterns
Uploaded source to S3: mcp_server_iam/source.zip
Updated CodeBuild project: bedrock-agentcore-mcp_server_iam-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.1s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 7.5s
🔄 DOWNLOAD_SOURCE started (total: 9s)
✅ DOWNLOAD_SOURCE completed in 1.1s
🔄 BUILD started (total: 10s)
✅ BUILD completed in 17.7s
🔄 POST_BUILD started (total: 27s)
✅ POST_BUILD completed in 5.3s
🔄 COMPLETED started (total: 33s)
✅ COMPLETED completed in 1.1s
🎉 CodeBuild completed successfully in 0m 33s
CodeBuild completed successfully
✅ CodeBuild pro

Launch completed ✓
Agent ARN: arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_iam-BH1RwGAovj
Agent ID: mcp_server_iam-BH1RwGAovj


In [9]:
agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime_iam/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server with inbound auth',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

✓ Agent ARN stored in Parameter Store

Configuration stored successfully!
Agent ARN: arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_iam-BH1RwGAovj


Creating Remote Testing Client

In [3]:
%%writefile mcp_client_remote.py       
import asyncio
import sys
import logging
import boto3
from boto3.session import Session
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from streamable_http_sigv4 import streamablehttp_client_with_sigv4


logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


def create_streamable_http_transport_sigv4(
    mcp_url: str, service_name: str, region: str
):
    """
    Create a streamable HTTP transport with AWS SigV4 authentication.

    This function creates an MCP client transport that uses AWS Signature Version 4 (SigV4)
    to authenticate requests. This is necessary because standard MCP clients don't natively
    support AWS IAM authentication, and this bridges that gap.

    Args:
        mcp_url (str): The URL of the MCP gateway endpoint
        service_name (str): The AWS service name for SigV4 signing (typically "bedrock-agentcore")
        region (str): The AWS region where the gateway is deployed

    Returns:
        StreamableHTTPTransportWithSigV4: A transport instance configured for SigV4 auth

    Example:
        >>> transport = create_streamable_http_transport_sigv4(
        ...     mcp_url=".../mcp",
        ...     service_name="bedrock-agentcore",
        ...     region="us-west-2"
        ... )
    """
    # Get AWS credentials from the current boto3 session
    # These credentials will be used to sign requests with SigV4
    session = boto3.Session()
    credentials = session.get_credentials()

    # Create and return the custom transport with SigV4 signing capability
    return streamablehttp_client_with_sigv4(
        url=mcp_url,
        credentials=credentials,
        service=service_name,
        region=region,
    )


def get_full_tools_list(client):
    """
    Retrieve the complete list of tools from an MCP client, handling pagination.

    MCP servers may return tools in paginated responses. This function handles the
    pagination automatically and returns all available tools in a single list.

    Args:
        client: An MCP client instance (from strands.tools.mcp.mcp_client.MCPClient)

    Returns:
        list: A complete list of all tools available from the MCP server

    Example:
        >>> mcp_client = MCPClient(lambda: create_transport())
        >>> all_tools = get_full_tools_list(mcp_client)
        >>> print(f"Found {len(all_tools)} tools")
    """
    more_tools = True
    tools = []
    pagination_token = None

    # Loop until we've fetched all pages
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)

        tools.extend(tmp_tools)

        # Check if there are more pages to fetch
        if tmp_tools.pagination_token is None:
            # No more pages - we're done
            more_tools = False
        else:
            # More pages exist - prepare to fetch the next one
            more_tools = True
            pagination_token = tmp_tools.pagination_token

    return tools


async def main():
    boto_session = Session()
    region = boto_session.region_name
    print(f"Using AWS region: {region}")

    ssm_client = boto3.client("ssm", region_name=region)

    agent_arn_response = ssm_client.get_parameter(
        Name="/mcp_server/runtime_iam/agent_arn"
    )
    agent_arn = agent_arn_response["Parameter"]["Value"]
    print(f"Retrieved Agent ARN: {agent_arn}")

    if not agent_arn:
        print("❌ Error: AGENT_ARN not found")
        sys.exit(1)

    encoded_arn = agent_arn.replace(":", "%3A").replace("/", "%2F")
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

    try:
        async with create_streamable_http_transport_sigv4(
            mcp_url=mcp_url, service_name="bedrock-agentcore", region=region
        ) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")

                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()

                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, "inputSchema") and tool.inputSchema:
                        properties = tool.inputSchema.get("properties", {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()

                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")

    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        import traceback

        print("\n🔍 Full error traceback:")
        traceback.print_exc()
        sys.exit(1)


if __name__ == "__main__":
    asyncio.run(main())

Overwriting mcp_client_remote.py


Testing Your Deployed MCP Server

In [4]:
print("Testing deployed MCP server...")
print("=" * 50)
# both works
# !uv run mcp_client_remote.py --region ap-southeast-2  --runtime-arn arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_iam-BH1RwGAovj
!uv run mcp_client_remote.py

Testing deployed MCP server...
Using AWS region: ap-southeast-2
2025-11-21 14:58:37,230 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
Retrieved Agent ARN: arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_iam-BH1RwGAovj
2025-11-21 14:58:37,567 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials

🔄 Initializing MCP session...
2025-11-21 14:58:38,253 - httpx - INFO - HTTP Request: POST https://bedrock-agentcore.ap-southeast-2.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aap-southeast-2%3A381491838394%3Aruntime%2Fmcp_server_iam-BH1RwGAovj/invocations?qualifier=DEFAULT "HTTP/1.1 200 OK"
2025-11-21 14:58:38,253 - mcp.client.streamable_http - INFO - Received session ID: 10766014-bcc5-432b-8e10-6b722d9a6abe
2025-11-21 14:58:38,254 - mcp.client.streamable_http - INFO - Negotiated protocol version: 2025-06-18
✓ MCP session initialized

🔄 Listing available tools...
2025

In [5]:
%%writefile invoke_mcp_tools.py
import asyncio
import sys
import os
import logging
import boto3
from boto3.session import Session
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from streamable_http_sigv4 import streamablehttp_client_with_sigv4

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


def create_streamable_http_transport_sigv4(
    mcp_url: str, service_name: str, region: str
):
    """
    Create a streamable HTTP transport with AWS SigV4 authentication.

    This function creates an MCP client transport that uses AWS Signature Version 4 (SigV4)
    to authenticate requests. This is necessary because standard MCP clients don't natively
    support AWS IAM authentication, and this bridges that gap.

    Args:
        mcp_url (str): The URL of the MCP gateway endpoint
        service_name (str): The AWS service name for SigV4 signing (typically "bedrock-agentcore")
        region (str): The AWS region where the gateway is deployed

    Returns:
        StreamableHTTPTransportWithSigV4: A transport instance configured for SigV4 auth

    Example:
        >>> transport = create_streamable_http_transport_sigv4(
        ...     mcp_url=".../mcp",
        ...     service_name="bedrock-agentcore",
        ...     region="us-west-2"
        ... )
    """
    # Get AWS credentials from the current boto3 session
    # These credentials will be used to sign requests with SigV4
    session = boto3.Session()
    credentials = session.get_credentials()

    # Create and return the custom transport with SigV4 signing capability
    return streamablehttp_client_with_sigv4(
        url=mcp_url,
        credentials=credentials,
        service=service_name,
        region=region,
    )


def get_full_tools_list(client):
    """
    Retrieve the complete list of tools from an MCP client, handling pagination.

    MCP servers may return tools in paginated responses. This function handles the
    pagination automatically and returns all available tools in a single list.

    Args:
        client: An MCP client instance (from strands.tools.mcp.mcp_client.MCPClient)

    Returns:
        list: A complete list of all tools available from the MCP server

    Example:
        >>> mcp_client = MCPClient(lambda: create_transport())
        >>> all_tools = get_full_tools_list(mcp_client)
        >>> print(f"Found {len(all_tools)} tools")
    """
    more_tools = True
    tools = []
    pagination_token = None

    # Loop until we've fetched all pages
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)

        tools.extend(tmp_tools)

        # Check if there are more pages to fetch
        if tmp_tools.pagination_token is None:
            # No more pages - we're done
            more_tools = False
        else:
            # More pages exist - prepare to fetch the next one
            more_tools = True
            pagination_token = tmp_tools.pagination_token

    return tools


async def main():
    boto_session = Session()
    region = boto_session.region_name
    print(f"Using AWS region: {region}")

    ssm_client = boto3.client("ssm", region_name=region)

    agent_arn_response = ssm_client.get_parameter(
        Name="/mcp_server/runtime_iam/agent_arn"
    )
    agent_arn = agent_arn_response["Parameter"]["Value"]
    print(f"Retrieved Agent ARN: {agent_arn}")

    if not agent_arn:
        print("❌ Error: AGENT_ARN not found")
        sys.exit(1)

    encoded_arn = agent_arn.replace(":", "%3A").replace("/", "%2F")
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

    try:
        async with create_streamable_http_transport_sigv4(
                mcp_url=mcp_url, service_name="bedrock-agentcore", region=region
        ) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")

                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()

                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")

                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)

                try:
                    print("\n➕ Testing add_numbers(5, 3)...")
                    add_result = await session.call_tool(
                        name="add_numbers", arguments={"a": 5, "b": 3}
                    )
                    print(f"   Result: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")

                try:
                    print("\n✖️  Testing multiply_numbers(4, 7)...")
                    multiply_result = await session.call_tool(
                        name="multiply_numbers", arguments={"a": 4, "b": 7}
                    )
                    print(f"   Result: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")

                try:
                    print("\n👋 Testing greet_user('Alice')...")
                    greet_result = await session.call_tool(
                        name="greet_user", arguments={"name": "Alice"}
                    )
                    print(f"   Result: {greet_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")

                print("\n✅ MCP tool testing completed!")

    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        import traceback

        print("\n🔍 Full error traceback:")
        traceback.print_exc()
        sys.exit(1)


if __name__ == "__main__":
    asyncio.run(main())

Writing invoke_mcp_tools.py


In [6]:
! uv run invoke_mcp_tools.py

Using AWS region: ap-southeast-2
2025-11-21 15:00:43,788 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
Retrieved Agent ARN: arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_iam-BH1RwGAovj
2025-11-21 15:00:44,144 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials

🔄 Initializing MCP session...
2025-11-21 15:00:44,707 - httpx - INFO - HTTP Request: POST https://bedrock-agentcore.ap-southeast-2.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aap-southeast-2%3A381491838394%3Aruntime%2Fmcp_server_iam-BH1RwGAovj/invocations?qualifier=DEFAULT "HTTP/1.1 200 OK"
2025-11-21 15:00:44,707 - mcp.client.streamable_http - INFO - Received session ID: a4157ea5-cf74-41a5-aea8-88bc8e9aa46a
2025-11-21 15:00:44,708 - mcp.client.streamable_http - INFO - Negotiated protocol version: 2025-06-18
✓ MCP session initialized

🔄 Listing available tools...
2025-11-21 15:00:44,872 - httpx - I

Cleanup (Optional)

In [12]:
# try:
#     ssm_client.delete_parameter(Name='/mcp_server/runtime_iam/agent_arn')
#     print("✓ Parameter Store parameter deleted")
# except ssm_client.exceptions.ParameterNotFound:
#     print("ℹ️  Parameter Store parameter not found")

In [13]:
# destroy_bedrock_agentcore(
#     config_path=Path(".bedrock_agentcore.yaml"),
#     agent_name=tool_name,
#     delete_ecr_repo=True
# )